In [ ]:
# Sistema e utilitários
import os
import gc
from copy import deepcopy
from datetime import datetime

# Instalações de pacotes
!pip install keras4torch
!pip install einops

# Bibliotecas principais
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Scikit-learn
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, QuantileTransformer

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Keras4Torch
import keras4torch
from keras4torch.callbacks import ModelCheckpoint, LRScheduler

# STab
import STab
from STab import *
from STab import mainmodel, LWTA, Gsoftmax

# Definição principal do modelo
MainModel = mainmodel.MainModel

In [6]:
from sklearn.datasets import load_iris

# Carregar Iris
iris = load_iris()
X = iris.data  # Apenas variáveis numéricas
y = iris.target

# Dividir em treino, validação e teste
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalizar
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train).astype(np.float32)
X_valid = scaler.transform(X_valid).astype(np.float32)
X_test  = scaler.transform(X_test).astype(np.float32)

# Ajustar rótulos para PyTorch
y_train = torch.tensor(y_train, dtype=torch.long)
y_valid = torch.tensor(y_valid, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Definir o modelo (sem categorias)
Or_model = mainmodel.MainModel(
    categories        = (),   # Sem features categóricas
    num_continuous    = 4,
    dim               = 16,
    dim_out           = 3,    # 3 classes (Iris)
    depth             = 2,
    heads             = 4,
    attn_dropout      = 0.1,
    ff_dropout        = 0.1,
    U                 = 2,
    cases             = 8,
)

# Criar o wrapper Num_Cat para gerenciar as entradas corretamente
no_model = Num_Cat(Or_model, num_number=4, classes=3, Sample_size=16)  # 4 features numéricas, 3 classes (Iris)

# Passar o modelo para keras4torch
model = keras4torch.Model(no_model).build([4])  # Agora só precisa da dimensão numérica


optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
model.compile(optimizer=optimizer, loss=F.cross_entropy, metrics=['accuracy'])

# Treinamento (passando apenas X numérico)
model.fit([X_train], y_train,  # Ajuste aqui: lista com X_train
          epochs=15, batch_size=16,
          validation_data=([X_valid], y_valid),  # Ajuste aqui
          verbose=2)

# Avaliação
test_logits = model.predict([X_test])  # Passar X_test como lista
test_preds = np.argmax(test_logits, axis=1)
accuracy = np.mean(test_preds == y_test.numpy())
print(f'Test Accuracy: {accuracy:.4f}')

report = classification_report(y_test.numpy(), test_preds, target_names=iris.target_names)
print(report)


Train on 105 samples, validate on 22 samples:
Epoch 1/15 - 3.8s - loss: 1.0388 - acc: 0.4571 - val_loss: 0.8196 - val_acc: 0.8636 - lr: 1e-03
Epoch 2/15 - 4.3s - loss: 0.7929 - acc: 0.7143 - val_loss: 0.6566 - val_acc: 0.9091 - lr: 1e-03
Epoch 3/15 - 3.2s - loss: 0.6391 - acc: 0.7810 - val_loss: 0.4962 - val_acc: 0.9545 - lr: 1e-03
Epoch 4/15 - 2.9s - loss: 0.5491 - acc: 0.8762 - val_loss: 0.4416 - val_acc: 0.9545 - lr: 1e-03
Epoch 5/15 - 2.6s - loss: 0.4763 - acc: 0.8476 - val_loss: 0.3947 - val_acc: 0.9545 - lr: 1e-03
Epoch 6/15 - 2.6s - loss: 0.4328 - acc: 0.8571 - val_loss: 0.3571 - val_acc: 0.9091 - lr: 1e-03
Epoch 7/15 - 2.5s - loss: 0.4867 - acc: 0.8286 - val_loss: 0.2750 - val_acc: 1.0000 - lr: 1e-03
Epoch 8/15 - 2.5s - loss: 0.3985 - acc: 0.8571 - val_loss: 0.2557 - val_acc: 1.0000 - lr: 1e-03
Epoch 9/15 - 2.4s - loss: 0.4345 - acc: 0.8667 - val_loss: 0.2295 - val_acc: 0.9545 - lr: 1e-03
Epoch 10/15 - 2.5s - loss: 0.3428 - acc: 0.9048 - val_loss: 0.2207 - val_acc: 1.0000 - lr: